In [4]:
import requests
from psycopg2 import connect, sql

In [5]:
conn_string = "dbname='etl_bites' user='' password='' host='localhost' port='5432'"

In [6]:
# Extract
def get_data_from_api(url):
    response = requests.get(url)
    return response.json()

In [7]:
posts_url = 'https://jsonplaceholder.typicode.com/posts'

In [8]:
users_url = "https://jsonplaceholder.typicode.com/users"

In [9]:
posts_data = get_data_from_api(posts_url)
users_data = get_data_from_api(users_url)

In [10]:
# Transform
def join_posts_and_users(posts, users):
    for post in posts:
        for user in users:
            if post['userId'] == user['id']:
                post['author'] = user['name']
    return posts

combined_data = join_posts_and_users(posts_data, users_data)

In [12]:
# Create tables in analytical DB
# This could also be done manually via a GUI (e.g. TablePlus) or with a SQL script
def execute_query_postgresql(conn_string, query):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

create_api_data_table = '''
CREATE TABLE IF NOT EXISTS api_data (
    post_id INTEGER NOT NULL,
    title TEXT NOT NULL,
    body TEXT NOT NULL,
    user_id INTEGER NOT NULL,
    author TEXT NOT NULL
);
'''

execute_query_postgresql(conn_string, create_api_data_table)

def insert_data_to_postgresql(conn_string, table_name, data):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            for item in data:
                query = sql.SQL("INSERT INTO {} (post_id, title, body, user_id, author) VALUES (%s, %s, %s, %s, %s)").format(sql.Identifier(table_name))
                cur.execute(query, (item['id'], item['title'], item['body'], item['userId'], item['author']))
        conn.commit()

table_name = "api_data"
insert_data_to_postgresql(conn_string, table_name, combined_data)


In [32]:
github_url = "https://api.github.com/repos/takeo/warden/stargazers/"
github_data = get_data_from_api(github_url)
github_data

[{'id': 1340328,
  'node_id': 'MDEwOlJlcG9zaXRvcnkxMzQwMzI4',
  'name': 'Brief',
  'full_name': 'takeo/Brief',
  'private': False,
  'owner': {'login': 'takeo',
   'id': 23,
   'node_id': 'MDQ6VXNlcjIz',
   'avatar_url': 'https://avatars.githubusercontent.com/u/23?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/takeo',
   'html_url': 'https://github.com/takeo',
   'followers_url': 'https://api.github.com/users/takeo/followers',
   'following_url': 'https://api.github.com/users/takeo/following{/other_user}',
   'gists_url': 'https://api.github.com/users/takeo/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/takeo/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/takeo/subscriptions',
   'organizations_url': 'https://api.github.com/users/takeo/orgs',
   'repos_url': 'https://api.github.com/users/takeo/repos',
   'events_url': 'https://api.github.com/users/takeo/events{/privacy}',
   'received_events_url': 'https://api.gith

In [25]:
github_url = "https://api.github.com/orgs/makersacademy/repos?all/1/50"
github_data = get_data_from_api(github_url)
github_data


[{'id': 8607845,
  'node_id': 'MDEwOlJlcG9zaXRvcnk4NjA3ODQ1',
  'name': 'minitest-example',
  'full_name': 'makersacademy/minitest-example',
  'private': False,
  'owner': {'login': 'makersacademy',
   'id': 3636186,
   'node_id': 'MDEyOk9yZ2FuaXphdGlvbjM2MzYxODY=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/3636186?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/makersacademy',
   'html_url': 'https://github.com/makersacademy',
   'followers_url': 'https://api.github.com/users/makersacademy/followers',
   'following_url': 'https://api.github.com/users/makersacademy/following{/other_user}',
   'gists_url': 'https://api.github.com/users/makersacademy/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/makersacademy/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/makersacademy/subscriptions',
   'organizations_url': 'https://api.github.com/users/makersacademy/orgs',
   'repos_url': 'https://api.github.com/use